# SQL проект. Анализ баз данных сервиса для чтения книг

# Описание исследования

**Цель исследования:** проанализировать базу данных сервиса для чтения книг по подписке.

**Входные данные:** База данных, в которой находится информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.

**План работы:**

1. Создать коннекцию к базам и получить данные;
2. Исследовать таблицы — вывести первые строки;
3. Выполнить задания, сделав по одному SQL-запросу для решения каждого задания:

   3.1. Посчитать, сколько книг вышло после 1 января 2000 года;
   
   3.2. Для каждой книги посчитать количество обзоров и среднюю оценку;
   
   3.3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так исключим из анализа брошюры;
   
   3.4. Определить автора с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками;
   
   3.5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.
   
5. Описать выводы по каждой из решённых задач.

**Данное исследование поможет сформулировать ценностное предложение для нового продукта.**

## Создаем коннекцию к базам и получаем данные

- **Загрузка библиотек**

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

- **Создаем коннекцию к базам** 

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Исследуем таблицы — выведем первые строки

In [3]:
# Выведем первые строки таблицы books
query = '''SELECT * FROM books LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
# Выведем первые строки таблицы authors
query = '''SELECT * FROM authors LIMIT 5'''
con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
# Выведем первые строки таблицы publishers
query = '''SELECT * FROM publishers LIMIT 5'''
con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
# Выведем первые строки таблицы ratings
query = '''SELECT * FROM ratings LIMIT 5'''
con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
# Выведем первые строки таблицы reviews
query = '''SELECT * FROM reviews LIMIT 5'''
con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


- **Описание данных**

**Таблица books**

Содержит данные о книгах:

- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

**Таблица authors**

Содержит данные об авторах:

- author_id — идентификатор автора;
- author — имя автора.

**Таблица publishers**

Содержит данные об издательствах:

- publisher_id — идентификатор издательства;
- publisher — название издательства;

**Таблица ratings**

Содержит данные о пользовательских оценках книг:

- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

**Таблица reviews**

Содержит данные о пользовательских обзорах на книги:

- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя пользователя, написавшего обзор;
- text — текст обзора.

## Выполняем задания, сделаем по одному SQL-запросу для решения каждого задания:

### Посчитаем, сколько книг вышло после 1 января 2000 года;

In [8]:
query = '''
SELECT COUNT(publisher_id)
FROM books
WHERE CAST(publication_date AS TIMESTAMP) >= CAST('2000-01-01' AS TIMESTAMP)
'''
pd.io.sql.read_sql(query, con = engine)

,count
0,821


**Вывод:**

После 01 января 2000 г. вышла 821 книга.

### Для каждой книги посчитаем количество обзоров и среднюю оценку;

In [13]:
query = '''                              
SELECT b.title,
       COUNT(DISTINCT rev.review_id), 
       AVG(r.rating)
FROM books b 
INNER JOIN ratings r ON r.book_id = b.book_id
LEFT JOIN reviews rev ON rev.book_id = b.book_id
GROUP BY b.title, b.book_id
ORDER BY COUNT(DISTINCT rev.review_id) DESC
;
''' 
pd.io.sql.read_sql(query, con = engine)

,title,count,avg
0,Twilight (Twilight #1),7,3.662500
1,Water for Elephants,6,3.977273
2,The Glass Castle,6,4.206897
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,The Natural Way to Draw,0,3.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,Essential Tales and Poems,0,4.000000


### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;

In [10]:
query = '''
SELECT p.publisher,
       COUNT(b.book_id)
FROM publishers p
JOIN books b ON p.publisher_id = b.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id
ORDER BY COUNT(b.book_id) DESC
LIMIT 1;
'''
pd.io.sql.read_sql(query, con = engine)

,publisher,count
0,Penguin Books,42


**Вывод:**

Издательство, которое выпустило наибольшее число книг толще 50 страниц - "Penguin Books".

### Определим автора с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками;

In [11]:
query = '''
SELECT a.author_id,
       a.author,
       AVG(r.rating),
       COUNT(r.rating)
FROM authors a
INNER JOIN books b ON b.author_id = a.author_id
INNER JOIN ratings r ON r.book_id = b.book_id
GROUP BY a.author_id, a.author
HAVING COUNT(r.rating) > 50 
ORDER BY AVG(r.rating) DESC
LIMIT 1;
'''
pd.io.sql.read_sql(query, con = engine)

,author_id,author,avg,count
0,236,J.K. Rowling/Mary GrandPré,4.288462,312


**Вывод:**

Автор с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками - J.K. Rowling.

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [12]:
query = '''
WITH avg AS
  (SELECT username,
          COUNT(review_id)
   FROM reviews
   WHERE username in
       (SELECT username
        FROM ratings
        GROUP BY username
        HAVING COUNT(rating_id)>50)
   GROUP BY username)
SELECT ROUND(SUM(COUNT)/COUNT(COUNT)) avg
FROM avg;
'''
pd.io.sql.read_sql(query, con = engine)

,avg
0,24.0


**Вывод:**

Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24.0.

## Общий вывод

- После 01 января 2000 г. вышла 821 книга;
- Для каждой книги посчитаем количество обзоров и среднюю оценку;
- Издательство, которое выпустило наибольшее число книг толще 50 страниц - "Penguin Books";
- Автор с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками - J.K. Rowling;
- Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24.0 шт.